In [2]:
import warnings
warnings.simplefilter('ignore')

# %matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import imblearn
import smote_variants as sv
import imbalanced_databases as imbd

In [3]:
clean_df = pd.read_csv('Resources/Final_Clean.csv')

In [4]:
clean_df.head()
clean_df = clean_df.drop(columns=['Unnamed: 0','Unnamed: 0.1'])


In [5]:
clean_df = clean_df.dropna(how = "any")

In [6]:
clean_df.head()

,NPA,year,Petitioner,FromZone,ToZone,Type,SPA,INNOV,Acres,Water,...,student_absenteeism,voters_participating,neighborhood_organizations,voter_participation,vacant_land,population_density,population,transit_ridership,street_connectivity,bicycle_friendliness
1,2,2019,Mark Heisig,O-2,R-4,NC,no,NO,0.33,CMUD,...,12.666667,756.333333,2.0,61.333333,4.3,5.0,2240.0,43.0,1.2,1.5
2,2,2014,"Eastway II Holdings, LLC",BD(CD),BD(CD),CD,SPA,NO,3.05,CMUD,...,11.500000,603.000000,2.0,54.000000,4.3,5.0,2240.0,43.0,1.2,1.5
3,2,2014,"Eastway II Holdings, LLC",B-1SCD,BD(CD),CD,no,NO,3.74,CMUD,...,11.500000,603.000000,2.0,54.000000,4.3,5.0,2240.0,43.0,1.2,1.5
4,2,2016,"Dr. Michael Berglass, DDS",UR-C(CD),UR-C(CD),CD,SPA,NO,0.44,CMUD,...,15.000000,1063.000000,2.0,76.000000,4.3,5.0,2240.0,43.0,1.2,1.5
5,3,2015,"1700 South, LLC",B-1,TOD-M,NC,no,NO,0.26,CMUD,...,5.721393,1354.000000,7.0,24.000000,2.5,7.0,7938.0,34.0,1.5,2.1


In [33]:
clean_df.dtypes
clean_df.count()


NPA                               1757
year                              1757
Petitioner                        1756
FromZone                          1757
ToZone                            1756
Type                              1756
SPA                               1755
INNOV                             1678
Acres                             1757
Water                             1735
Sewer                             1736
CityLimit                         1748
CCW                               1715
ServiceArea                       1712
LuseStaff                         1705
UrbanDesign                       1713
RezoneStaff                       1662
Received                          1751
Approved                          1600
Decision                          1754
births_to_adolescents             1757
grocery_proximate_units           1757
prenatal_care                     1757
grocery_proximity                 1757
low_birthweight                   1757
public_health_insurance  

In [7]:
column_names = list(clean_df.columns.values)
print(column_names)
len(column_names)

['NPA', 'year', 'Petitioner', 'FromZone', 'ToZone', 'Type', 'SPA', 'INNOV', 'Acres', 'Water', 'Sewer', 'CityLimit', 'CCW', 'ServiceArea', 'LuseStaff', 'UrbanDesign', 'RezoneStaff', 'Received', 'Approved', 'Decision', 'births_to_adolescents', 'grocery_proximate_units', 'prenatal_care', 'grocery_proximity', 'low_birthweight', 'public_health_insurance', 'age_of_death', 'impervious_surface', 'water_consumption', 'neighborhood_school_attendance', 'highschool_graduation_rate', 'proficiency_elementary_school', 'proficiency_middle_school', 'proficiency_high_school', 'student_absenteeism', 'voters_participating', 'neighborhood_organizations', 'voter_participation', 'vacant_land', 'population_density', 'population', 'transit_ridership', 'street_connectivity', 'bicycle_friendliness']


44

In [8]:
#catagorical_columns drop 'Decision', which will be our Target
catagorical_columns = ['NPA', 'year', 'Petitioner', 'FromZone', 'ToZone', 'Type', 'SPA', 'INNOV', 'Water', 'Sewer', 'CityLimit', 'CCW', 'ServiceArea', 'LuseStaff', 'UrbanDesign', 'RezoneStaff', 'Received', 'Approved']
numberic_columns = ['Acres', 'births_to_adolescents', 'grocery_proximate_units', 'prenatal_care', 'grocery_proximity',
                  'low_birthweight', 'public_health_insurance', 'age_of_death', 'impervious_surface',
                  'water_consumption', 'neighborhood_school_attendance', 'highschool_graduation_rate', 
                   'proficiency_elementary_school', 'proficiency_middle_school', 'proficiency_high_school',
                   'student_absenteeism', 'voters_participating', 'neighborhood_organizations',
                   'voter_participation', 'vacant_land', 'population_density', 'population', 'transit_ridership',
                   'street_connectivity', 'bicycle_friendliness']
all_columns = ['NPA', 'year', 'Petitioner', 'FromZone', 'ToZone', 'Type', 'SPA', 'INNOV', 'Acres', 'Water', 'Sewer', 'CityLimit', 'CCW', 'ServiceArea', 'LuseStaff', 'UrbanDesign', 'RezoneStaff', 'Received', 'Approved', 'Decision', 'births_to_adolescents', 'grocery_proximate_units', 'prenatal_care', 'grocery_proximity', 'low_birthweight', 'public_health_insurance', 'age_of_death', 'impervious_surface', 'water_consumption', 'neighborhood_school_attendance', 'highschool_graduation_rate', 'proficiency_elementary_school', 'proficiency_middle_school', 'proficiency_high_school', 'student_absenteeism', 'voters_participating', 'neighborhood_organizations', 'voter_participation', 'vacant_land', 'population_density', 'population', 'transit_ridership', 'street_connectivity', 'bicycle_friendliness']



In [9]:
print(len(catagorical_columns))
print(len(numberic_columns))
print("______")
print(len(catagorical_columns)+len(numberic_columns))
print("______")
print(len(clean_df.columns))

18
25
______
43
______
44


In [10]:
y = clean_df[['Decision']]
y.fillna("UKN", inplace=True)

In [11]:
clean_drop_df = clean_df.drop(['Decision'], axis=1)

In [12]:
#Dummy Encode Catagorical Data

X = pd.get_dummies(clean_drop_df, columns=catagorical_columns)
X.head()

,Acres,births_to_adolescents,grocery_proximate_units,prenatal_care,grocery_proximity,low_birthweight,public_health_insurance,age_of_death,impervious_surface,water_consumption,...,Approved_2020/02/17 00:00:00+00,Approved_2020/03/16 00:00:00+00,Approved_2020/05/18 00:00:00+00,Approved_2020/05/18 16:02:33+00,Approved_2020/05/18 16:18:00+00,Approved_2020/05/18 16:38:34+00,Approved_2020/05/18 16:38:48+00,Approved_2020/05/18 16:46:28+00,Approved_2020/05/18 16:48:49+00,Approved_2020/06/15 00:00:00+00
1,0.33,3.166667,204.333333,70.466667,19.0,17.733333,13.0,68.666667,22.2,112.333333,...,0,0,0,0,0,0,0,0,0,0
2,3.05,1.400000,204.000000,75.700000,19.0,21.600000,13.0,68.000000,22.2,112.000000,...,0,0,0,0,0,0,0,0,0,0
3,3.74,1.400000,204.000000,75.700000,19.0,21.600000,13.0,68.000000,22.2,112.000000,...,0,0,0,0,0,0,0,0,0,0
4,0.44,6.700000,205.000000,60.000000,19.0,10.000000,13.0,70.000000,22.2,113.000000,...,0,0,0,0,0,0,0,0,0,0
5,0.26,3.375000,3118.000000,82.300000,66.0,8.250000,4.0,70.000000,37.3,164.473684,...,0,0,0,0,0,0,0,0,0,0


In [14]:
oversampler = sv.distance_SMOTE()
#x_samp , y_samp= oversampler.sample(X,y)

In [17]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
steps = [('over', SMOTE()), ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)

In [18]:
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
# define dataset
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
                           n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=1)
# define pipeline
steps = [('over', SMOTE()), ('model', DecisionTreeClassifier())]
pipeline = Pipeline(steps=steps)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.821


In [ ]:
#Chosing an oversampler supporting multiclass oversampling 
#oversampler = sv.MulticlassOversampling(sv.distance_SMOTE())

In [ ]:
#x_samp , y_samp= oversampler.sample(X,y)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
label_encoder.fit(y)
label_encoder.classes_

In [ ]:
encoded_y = label_encoder.transform(y_samp)

In [ ]:
print(encoded_y)

In [ ]:
print(x_samp.ahape, encoded_y.shape)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_samp, encoded_y, random_state=42)

#print(X_train.head())
#print(y_train[0:5])

In [ ]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt


In [ ]:
# Loop through different k values to see which has the highest accuracy
# Note: We only use odd numbers because we don't want any ties
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy Score")
plt.show()

In [ ]:
# Note that k: 5 provides the best accuracy where the classifier starts to stablize
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
print('k=5 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))